In [2]:
import os

import numpy as np 
import pandas as pd 

from ast import literal_eval

import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
df_credits = pd.read_csv(os.path.join('data', 'original', 'credits.csv'))
df_keywords = pd.read_csv(os.path.join('data', 'original', 'keywords.csv'))
df_links = pd.read_csv(os.path.join('data', 'original', 'links.csv'))
df_movies = pd.read_csv(os.path.join('data', 'original', 'movies_metadata.csv'))
df_ratings = pd.read_csv(os.path.join('data', 'original', 'ratings.csv'))

C:\Users\PREDATOR HELIOS 300\AppData\Local\Temp\ipykernel_10016\1824330828.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies = pd.read_csv(os.path.join('data', 'movies_metadata.csv'))


In [4]:
print("Columns in credits dataset:", df_credits.columns.tolist())
print("Columns in keywords dataset:", df_keywords.columns.tolist())
print("Columns in links dataset:", df_links.columns.tolist())
print("Columns in movies dataset:", df_movies.columns.tolist())
print("Columns in ratings dataset:", df_ratings.columns.tolist())

Columns in credits dataset: ['cast', 'crew', 'id']
Columns in keywords dataset: ['id', 'keywords']
Columns in links dataset: ['movieId', 'imdbId', 'tmdbId']
Columns in movies dataset: ['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id', 'imdb_id', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'video', 'vote_average', 'vote_count']
Columns in ratings dataset: ['userId', 'movieId', 'rating', 'timestamp']


### **Check Linkage**

This section checks the linkage between the datasets and filters based on common movie IDs

In [5]:
## Keywords x Credits x Links
# convert IDs to string type and remove duplicates

df_credits_id = df_credits['id'].astype(str).drop_duplicates()
df_keywords_id = df_keywords['id'].astype(str).drop_duplicates()
df_movies_id = df_movies['id'].astype(str).drop_duplicates()
print("Shape of Credits ID with no duplicates:", df_credits_id.shape)
print("Shape of Keywords ID with no duplicates:", df_keywords_id.shape)
print("Shape of Movies ID with no duplicates:", df_movies_id.shape)

# convert IDs to string type in links dataset and remove duplicates

df_links['imdbId'] = df_links['imdbId'].astype(str)
df_links['tmdbId'] = df_links['tmdbId'].astype(str)
df_links['tmdbId'] = df_links['tmdbId'].str.split('.').str[0]
df_links['movieId'] = df_links['movieId'].astype(str)
df_links = df_links.drop_duplicates()
print("Shape of Links with no duplicates:", df_links.shape)

# merge df_credits_id, df_keywords_id, df_movies_id, and df_links on tmdbId to get common movie IDs

df_tmdbId = pd.merge(df_credits_id, df_keywords_id, left_on='id', right_on='id', how='inner')
df_tmdbId = pd.merge(df_tmdbId['id'], df_movies_id, left_on='id', right_on='id', how='inner')
df_tmdbId = pd.merge(df_tmdbId['id'], df_links[['tmdbId', 'movieId']], left_on='id', right_on='tmdbId', how='inner')

## Ratings x Links
# remove duplicates and unnecessary columns

df_tmdbId = df_tmdbId.drop(columns=['tmdbId']).drop_duplicates()
print("-- Shape of Merged Credits, Keywords, Movies, and Links (TMDB ID):", df_tmdbId.shape)

# convert movieId to string type in ratings dataset and remove duplicates

df_ratings_id = df_ratings['movieId'].astype(str).drop_duplicates()
print("Shape of Ratings ID with no duplicates:", df_ratings_id.shape)

# merge df_ratings_id with df_links to get common movie IDs

df_movieId = pd.merge(df_ratings_id, df_links[['movieId']], left_on='movieId', right_on='movieId', how='inner')
print("-- Shape of Merged Ratings and Links (Movie ID):", df_movieId.shape)

## Final merge to get all common movie IDs

df_all_id = pd.merge(df_tmdbId, df_movieId, left_on='movieId', right_on='movieId', how='inner')
print("-- Shape of All Common Movie IDs (Merged TMDB and Movie ID):", df_all_id.shape)

Shape of Credits ID with no duplicates: (45432,)
Shape of Keywords ID with no duplicates: (45432,)
Shape of Movies ID with no duplicates: (45436,)
Shape of Links with no duplicates: (45843, 3)
-- Shape of Merged Credits, Keywords, Movies, and Links (TMDB ID): (45462, 2)
Shape of Ratings ID with no duplicates: (45115,)
-- Shape of Merged Ratings and Links (Movie ID): (45115, 1)
-- Shape of All Common Movie IDs (Merged TMDB and Movie ID): (44740, 2)


### **Filter with Movies Dataset**

1. Remove Duplicates and Merge with Common IDs

In [6]:
print("-- Original Shape of Movies:", df_movies.shape)

# Exclude specified columns from duplicate analysis
columns_to_exclude = ['belongs_to_collection', 'genres', 'production_companies', 'production_countries', 'spoken_languages']
columns_to_check = [col for col in df_movies.columns if col not in columns_to_exclude]
duplicate_count = df_movies.duplicated(subset=columns_to_check).sum()
print("Number of duplicated rows in Movies: ", duplicate_count)

# Remove duplicates based on the specified columns
df_movies = df_movies.drop_duplicates(subset=columns_to_check, keep='first')
print("Shape of Movies after dropping duplicates:", df_movies.shape)

# Merge filtered movies with All Common Movie IDs to retain only common movie IDs
df_movies['id'] = df_movies['id'].astype(str)
df_movies_filtered = pd.merge(df_all_id, df_movies, left_on='id', right_on='id', how='inner')
print("-- Shape of Merged Filtered Movies and All Common Movie IDs (Filtered Movies):", df_movies_filtered.shape)

-- Original Shape of Movies: (45466, 24)
Number of duplicated rows in Movies:  13
Shape of Movies after dropping duplicates: (45453, 24)
-- Shape of Merged Filtered Movies and All Common Movie IDs (Filtered Movies): (44774, 25)


2. English Movies Only

In [7]:
print("-- Original Shape of Filtered Movies:", df_movies_filtered.shape)

# Filter rows where original_language is 'en'
df_movies_filtered = df_movies_filtered[df_movies_filtered['original_language']=='en']

# Filter rows where English ('iso_639_1': 'en') is present in the spoken_languages list
# Ensure the spoken_languages column is parsed as a list of dictionaries
df_movies_filtered['spoken_languages'] = df_movies_filtered['spoken_languages'].fillna('[]').apply(literal_eval)
df_movies_filtered = df_movies_filtered[df_movies_filtered['spoken_languages'].apply(lambda x: any(lang['iso_639_1'] == 'en' for lang in x))]

print("-- Shape of Filtered Movies after filtering for English spoken language (Filtered Movies):", df_movies_filtered.shape)

# Merge filtered movies with All Common Movie IDs to retain only common movie IDs
df_movies_filtered_id = df_movies_filtered['id'].astype(str).drop_duplicates()
df_all_id_filtered = pd.merge(df_all_id, df_movies_filtered_id, left_on='id', right_on='id', how='inner')
print("-- Shape of Merged Filtered Movies and All Common Movie IDs (Filtered IDs):", df_all_id_filtered.shape)

-- Original Shape of Filtered Movies: (44774, 25)
-- Shape of Filtered Movies after filtering for English spoken language (Filtered Movies): (26437, 25)
-- Shape of Merged Filtered Movies and All Common Movie IDs (Filtered IDs): (26415, 2)


3. Newer than 1990

In [8]:
print("-- Original Shape of Filtered Movies:", df_movies_filtered.shape)

# Convert 'release_date' column to datetime type
df_movies_filtered['release_date'] = pd.to_datetime(df_movies_filtered['release_date'], errors='coerce')
print("Initial minimum release date of Filtered Movies: ", {df_movies_filtered['release_date'].min()})
print("Initial maximum release date of Filtered Movies: ", {df_movies_filtered['release_date'].max()})

# Filter movies released after December 31, 1989
df_movies_filtered = df_movies_filtered[df_movies_filtered['release_date'] > '1989-12-31']
print("-- Shape of Filtered Movies after filtering for release date > 1989-12-31 (Filtered Movies):", df_movies_filtered.shape)
print("Final minimum release date of Filtered Movies: ", {df_movies_filtered['release_date'].min()})

# Merge filtered movies with All Common Movie IDs to retain only common movie IDs
df_movies_id_filtered = df_movies_filtered['id'].astype(str).drop_duplicates()
df_all_id_filtered = pd.merge(df_all_id_filtered, df_movies_id_filtered, left_on='id', right_on='id', how='inner')
print("-- Shape of Merged Filtered Movies and All Common Movie IDs (Filtered IDs):", df_all_id_filtered.shape)

-- Original Shape of Filtered Movies: (26437, 25)
Initial minimum release date of Filtered Movies:  {Timestamp('1903-01-21 00:00:00')}
Initial maximum release date of Filtered Movies:  {Timestamp('2020-12-16 00:00:00')}
-- Shape of Filtered Movies after filtering for release date > 1989-12-31 (Filtered Movies): (17167, 25)
Final minimum release date of Filtered Movies:  {Timestamp('1990-01-01 00:00:00')}
-- Shape of Merged Filtered Movies and All Common Movie IDs (Filtered IDs): (17155, 2)


4. Released Status Only

In [9]:
print("-- Original Shape of Filtered Movies:", df_movies_filtered.shape)

# Analyze 'status' column
status_counts = df_movies_filtered['status'].value_counts()
print("\nStatus and their frequency:")
display(status_counts)
print("\n")

# Filter movies with 'Released' status
df_movies_filtered = df_movies_filtered[df_movies_filtered['status'] == 'Released']
print("-- Shape of Filtered Movies after filtering for 'Released' status (Filtered Movies):", df_movies_filtered.shape)

# Merge filtered movies with All Common Movie IDs to retain only common movie IDs
df_movies_id_filtered = df_movies_filtered['id'].astype(str).drop_duplicates()
df_all_id_filtered = pd.merge(df_all_id_filtered, df_movies_id_filtered, left_on='id', right_on='id', how='inner')
print("-- Shape of Merged Filtered Movies and All Common Movie IDs (Filtered IDs):", df_all_id_filtered.shape)

-- Original Shape of Filtered Movies: (17167, 25)

Status and their frequency:


status
Released           17012
Post Production       63
Rumored               57
In Production         10
Planned                9
Name: count, dtype: int64



-- Shape of Filtered Movies after filtering for 'Released' status (Filtered Movies): (17012, 25)
-- Shape of Merged Filtered Movies and All Common Movie IDs (Filtered IDs): (17000, 2)


5. Remove Empty Overviews

In [10]:
print("-- Original Shape of Filtered Movies:", df_movies_filtered.shape)

# Find rows where 'overview' is empty or NaN
empty_overview_rows = df_movies_filtered[df_movies_filtered['overview'].isna() | (df_movies_filtered['overview'].str.strip() == '')]
print("Shape of rows with empty overviews in Filtered Movies:", empty_overview_rows.shape[0])

# Remove rows with empty or NaN overviews
df_movies_filtered = df_movies_filtered[~(df_movies_filtered['overview'].isna() | (df_movies_filtered['overview'].str.strip() == ''))]
print("-- Shape of Filtered Movies after removing rows with empty or NaN overviews (Filtered Movies):", df_movies_filtered.shape)

# Merge filtered movies with All Common Movie IDs to retain only common movie IDs
df_movies_id_filtered = df_movies_filtered['id'].astype(str).drop_duplicates()
df_all_id_filtered = pd.merge(df_all_id_filtered, df_movies_id_filtered, left_on='id', right_on='id', how='inner')
print("-- Shape of Merged Filtered Movies and All Common Movie IDs (Filtered IDs):", df_all_id_filtered.shape)

-- Original Shape of Filtered Movies: (17012, 25)
Shape of rows with empty overviews in Filtered Movies: 30
-- Shape of Filtered Movies after removing rows with empty or NaN overviews (Filtered Movies): (16982, 25)
-- Shape of Merged Filtered Movies and All Common Movie IDs (Filtered IDs): (16970, 2)


6. Remove some rows

In [11]:
print("-- Original Shape of Filtered Movies:", df_movies_filtered.shape)

# Drop these rows because of issues found during analysis
df_movies_filtered = df_movies_filtered[df_movies_filtered['id'] != "348858"]
df_movies_filtered = df_movies_filtered[df_movies_filtered['id'] != "4912"]
print("-- Shape of Filtered Movies after dropping specified rows (Filtered Movies):", df_movies_filtered.shape)

# Merge filtered movies with df_all_id to retain only common movie IDs
df_movies_id_filtered = df_movies_filtered['id'].astype(str).drop_duplicates()
df_all_id_filtered = pd.merge(df_all_id_filtered, df_movies_id_filtered, left_on='id', right_on='id', how='inner')
print("-- Shape of Merged Filtered Movies and All Common Movie IDs (Filtered IDs):", df_all_id_filtered.shape)

-- Original Shape of Filtered Movies: (16982, 25)
-- Shape of Filtered Movies after dropping specified rows (Filtered Movies): (16978, 25)
-- Shape of Merged Filtered Movies and All Common Movie IDs (Filtered IDs): (16968, 2)


7. Remove duplicate Ids in link

In [12]:
# Find rows with duplicate 'movieId'
duplicate_movieId_rows = df_all_id_filtered[df_all_id_filtered.duplicated(subset=['movieId'], keep=False)]

print("Rows with duplicate 'movieId' in Filtered IDs:", duplicate_movieId_rows.shape)
display(duplicate_movieId_rows)

# Find rows with duplicate 'id'
duplicate_id_rows = df_all_id_filtered[df_all_id_filtered.duplicated(subset=['id'], keep=False)]

print("\nRows with duplicate 'id' in Filtered IDs:", duplicate_id_rows.shape)
display(duplicate_id_rows)

Rows with duplicate 'movieId' in Filtered IDs: (0, 2)


,id,movieId



Rows with duplicate 'id' in Filtered IDs: (16, 2)


,id,movieId
1484,84198,2679
1485,84198,103208
2416,15028,5264
2417,15028,144426
3932,69234,31026
3933,69234,123107
4248,14788,38188
4249,14788,55132
4598,77221,47237
4599,77221,102174


7. Check df_movies

In [13]:
print("-- Original Shape of Filtered Movies:", df_movies_filtered.shape)
df_movies_filtered = df_movies_filtered.drop(columns=['movieId'])

# Exclude specified columns from duplicate analysis
columns_to_exclude = ['belongs_to_collection', 'genres', 'production_companies', 'production_countries', 'spoken_languages']
columns_to_check = [col for col in df_movies_filtered.columns if col not in columns_to_exclude]
duplicate_count = df_movies_filtered.duplicated(subset=columns_to_check).sum()
print(f"Number of duplicated rows in Filtered Movies: {duplicate_count}")

# Remove duplicates based on the specified columns
df_movies_filtered = df_movies_filtered.drop_duplicates(subset=columns_to_check, keep='first')
print("-- Shape of Filtered Movies after dropping duplicates (Filtered Movies):", df_movies_filtered.shape)

-- Original Shape of Filtered Movies: (16978, 25)
Number of duplicated rows in Filtered Movies: 13
-- Shape of Filtered Movies after dropping duplicates (Filtered Movies): (16965, 24)


In [14]:
# Check for duplicated 'id' rows and analyze differences
duplicate_count = df_movies_filtered['id'].duplicated(keep=False).sum()
print(f"Number of duplicated 'id' rows in Filtered Movies: {duplicate_count}")
duplicated_rows = df_movies_filtered[df_movies_filtered['id'].duplicated(keep=False)]

# Iterate through each duplicated id
for duplicate_id in duplicated_rows['id'].unique():
    # Filter rows with the same id
    rows_with_duplicate_id = duplicated_rows[duplicated_rows['id'] == duplicate_id]
    
    # Drop excluded columns and check if the remaining columns differ
    rows_to_compare = rows_with_duplicate_id[columns_to_check]
    differing_columns = rows_to_compare.nunique()[rows_to_compare.nunique() > 1].index.tolist()
    
    if differing_columns:
        print(f"Duplicated id {duplicate_id} has differing values in columns: {differing_columns}")
    else:
        print(f"Duplicated id {duplicate_id} has no differing values (excluding {columns_to_exclude}).")

Number of duplicated 'id' rows in Filtered Movies: 10


Duplicated id 84198 has differing values in columns: ['popularity']
Duplicated id 15028 has differing values in columns: ['popularity', 'vote_count']
Duplicated id 69234 has differing values in columns: ['popularity']
Duplicated id 14788 has differing values in columns: ['popularity']
Duplicated id 77221 has differing values in columns: ['popularity']


In [ ]:
print("-- Original Shape of Filtered Movies:", df_movies_filtered.shape)

# Ensure 'popularity' and 'vote_count' columns are numeric
df_movies_filtered['popularity'] = pd.to_numeric(df_movies_filtered['popularity'], errors='coerce')
df_movies_filtered['vote_count'] = pd.to_numeric(df_movies_filtered['vote_count'], errors='coerce')

# Group by 'id' and calculate the mean for the specified columns
columns_to_average = ['popularity', 'vote_count']
df_movies_filtered = (
    df_movies_filtered.groupby('id', as_index=False)
    .agg({col: 'mean' for col in columns_to_average})
    .round({'vote_count': 0})  # Round 'vote_count' to the nearest integer
    .merge(df_movies_filtered.drop(columns=columns_to_average).drop_duplicates(subset=['id']), on='id', how='left')
)

# Ensure 'vote_count' is an integer
df_movies_filtered['vote_count'] = df_movies_filtered['vote_count'].astype(int)

# Merge filtered movies with All Common Movie IDs to retain only common movie IDs
df_movies_id_filtered = df_movies_filtered['id'].astype(str).drop_duplicates()
print("-- Shape of Filtered Movies after handling duplicates (Filtered Movies):", df_movies_filtered.shape)
df_all_id_filtered = pd.merge(df_all_id_filtered, df_movies_id_filtered, left_on='id', right_on='id', how='inner')
print("-- Shape of Merged Filtered Movies and All Common Movie IDs (Filtered IDs):", df_all_id_filtered.shape)

-- Original Shape of Filtered Movies: (16965, 24)
Shape of Filtered Movies after handling duplicates (Filtered Movies): (16960, 24)
Shape of Merged Filtered Movies and All Common Movie IDs (Filtered IDs): (16968, 2)


7. Check df_ratings

In [16]:
print("-- Original Shape of Ratings:", df_ratings.shape)

df_ratings_filtered = df_ratings.copy()
df_ratings_filtered['movieId'] = df_ratings_filtered['movieId'].astype(str)

# Filter df_ratings for movieIds in duplicate_id_rows
duplicate_movieIds = duplicate_id_rows['movieId'].unique()
filtered_ratings = df_ratings_filtered[df_ratings_filtered['movieId'].isin(duplicate_movieIds)]

# Count the number of rows for each movieId
print("Number of rows in Ratings for each duplicate movieId:")
movieId_counts = filtered_ratings['movieId'].value_counts()
display(movieId_counts)

-- Original Shape of Ratings: (26024289, 4)
Number of rows in Ratings for each duplicate movieId:


movieId
5264      798
105769    213
38188     174
31026     133
2679       61
144426     33
70507      22
55132      19
103208     19
123107     18
102174     15
70545      15
116752      5
47237       4
106866      4
98572       1
Name: count, dtype: int64

In [17]:
print("-- Original Shape of Ratings:", df_ratings.shape)

# Create a mapping of id to the first instance of movieId in duplicate_id_rows
id_to_movieId_map = duplicate_id_rows.drop_duplicates(subset=['id']).set_index('id')['movieId'].to_dict()

# Create a mapping of movieId to id from duplicate_id_rows
movieId_to_id_map = duplicate_id_rows.set_index('movieId')['id'].to_dict()

# Map movieId in df_ratings_filtered to id using movieId_to_id_map
df_ratings_filtered['id'] = df_ratings_filtered['movieId'].map(movieId_to_id_map)

# Update movieId in df_ratings_filtered using id_to_movieId_map
df_ratings_filtered['movieId'] = df_ratings_filtered['id'].map(id_to_movieId_map).fillna(df_ratings_filtered['movieId'])

# Drop the temporary 'id' column
df_ratings_filtered = df_ratings_filtered.drop(columns=['id'])

# Filter df_ratings for movieIds in duplicate_id_rows
duplicate_movieIds = duplicate_id_rows['movieId'].unique()
df_ratings_filtered['movieId'] = df_ratings_filtered['movieId'].astype(str)
filtered_ratings = df_ratings_filtered[df_ratings_filtered['movieId'].isin(duplicate_movieIds)]

# Count the number of rows for each movieId
movieId_counts = filtered_ratings['movieId'].value_counts()

# Display the counts
print("Number of rows in Ratings for each duplicate movieId:")
display(movieId_counts)

print("-- Shape of Filtered Ratings after handling duplicates:", df_ratings_filtered.shape)

-- Original Shape of Ratings: (26024289, 4)
Number of rows in Ratings for each duplicate movieId:


movieId
5264     831
98572    214
38188    193
31026    151
2679      80
70507     26
70545     20
47237     19
Name: count, dtype: int64

-- Shape of Filtered Ratings after handling duplicates: (26024289, 4)


7. Check df_all_id_filtered

In [18]:
print("-- Original Shape of Filtered IDs:", df_all_id_filtered.shape)
# Remove duplicates by keeping only the first instance of movieId for each id
df_all_id_filtered = df_all_id_filtered.drop_duplicates(subset=['id'], keep='first')
print("-- Shape of Filtered IDs after removing duplicates by 'id':", df_all_id_filtered.shape)

-- Original Shape of Filtered IDs: (16968, 2)
-- Shape of Filtered IDs after removing duplicates by 'id': (16960, 2)


### **Feature Engg with Credits Dataset**

1. Merge with Common IDs

In [19]:
print("-- Original Shape of Credits:", df_credits.shape)
df_credits_new = df_credits.copy()
df_credits_new['cast'] = df_credits_new['cast'].fillna('[]').apply(literal_eval)
df_credits_new['crew'] = df_credits_new['crew'].fillna('[]').apply(literal_eval)

df_credits_new['id'] = df_credits_new['id'].astype(str)
df_credits_new = pd.merge(df_all_id_filtered[['id']].drop_duplicates(keep='first'), df_credits_new, left_on='id', right_on='id', how='inner')
print("-- Shape of Merged Credits and All Common Movie IDs (New Credits):", df_credits_new.shape)

-- Original Shape of Credits: (45476, 3)
-- Shape of Merged Credits and All Common Movie IDs (New Credits): (16974, 3)


2. Processing Cast Column (Top 3 Cast Only)

In [20]:
def get_key_cast(cast_list):
    """
    Processes the 'cast' column by extracting names, keeping only the first 3,
    and ensuring the input is a list.
    """
    if isinstance(cast_list, list):
        # Extract the 'name' field and keep only the first 3 names
        return [i['name'] for i in cast_list][:3]
    return []

df_credits_new['cast'] = df_credits_new['cast'].apply(get_key_cast)
display(df_credits_new[['id','cast']].head())

,id,cast
0,862,"[Tom Hanks, Tim Allen, Don Rickles]"
1,8844,"[Robin Williams, Jonathan Hyde, Kirsten Dunst]"
2,15602,"[Walter Matthau, Jack Lemmon, Ann-Margret]"
3,31357,"[Whitney Houston, Angela Bassett, Loretta Devine]"
4,11862,"[Steve Martin, Diane Keaton, Martin Short]"


2. Processing Crew Column (Writers and Directors Only)

In [21]:
# High-impact job titles
DIRECTING_JOBS = {'Director', 'Co-Director', 'Special Guest Director'}
WRITING_JOBS = {
    'Screenplay', 'Writer', 'Co-Writer', 'Story', 'Author', 
    'Novel', 'Book', 'Short Story', 'Comic Book', 'Theatre Play', 'Characters'
}

# Hard limit for directors and writers
DIRECTOR_LIMIT = 3
WRITER_LIMIT = 7

def get_key_crew(crew_list):
    """
    Extracts key crew members with limits to handle outliers like anthologies.
    """
    directors = []
    priority_directors = []
    
    temp_writers = []
    priority_writers = []
    screenplay_writers = []
    
    # --- Extract all potential candidates first ---
    for member in crew_list:
        job = member.get('job')
        name = member['name']
        
        if job in DIRECTING_JOBS:
            directors.append(name)
            if job == 'Director':
                priority_directors.append(name)
        elif job in WRITING_JOBS:
            temp_writers.append(name)
            if job in {'Screenplay', 'Author', 'Novel', 'Book'}:
                priority_writers.append(name)
            if job == 'Screenplay':
                screenplay_writers.append(name)
                
    # --- Apply the filtering rules ---
    
    # WRITER CONDITIONS
    if len(temp_writers) > WRITER_LIMIT and len(priority_writers) > WRITER_LIMIT and len(screenplay_writers) > 0:
        # If all lists are too long, use only those who worked on the screenplay 
        final_writers = list(set(screenplay_writers))[:WRITER_LIMIT]
    elif len(temp_writers) > WRITER_LIMIT and len(priority_writers) > 0:
        # If the main list is too long, use those in the priority list (screenplay, author, novel, book)
        final_writers = list(set(priority_writers))[:WRITER_LIMIT]
    else:
        # Otherwise, use all the main list
        final_writers = list(set(temp_writers))[:WRITER_LIMIT]
            
    # DIRECTOR CONDITIONS
    if len(directors) > DIRECTOR_LIMIT and len(priority_directors) > 0:
        # If all directors exceed limit, use only primary directors
        final_directors = list(set(priority_directors))[:DIRECTOR_LIMIT]
    elif len(directors) > DIRECTOR_LIMIT:
        # This case handles when there are many co-directors but no primary director
        final_directors = [] 
    else:
        # Otherwise, use all directors
        final_directors = list(set(directors))[:DIRECTOR_LIMIT]
            
    return final_directors, final_writers

df_credits_new[['directors', 'writers']] = df_credits_new['crew'].apply(get_key_crew).apply(pd.Series)
df_credits_new = df_credits_new.drop(columns=['crew'])
display(df_credits_new.head())

,id,cast,directors,writers
0,862,"[Tom Hanks, Tim Allen, Don Rickles]",[John Lasseter],"[Joel Cohen, Andrew Stanton, Joss Whedon, Alec Sokolow]"
1,8844,"[Robin Williams, Jonathan Hyde, Kirsten Dunst]",[Joe Johnston],"[Chris van Allsburg, Greg Taylor, Jim Strain, Jonathan Hensleigh]"
2,15602,"[Walter Matthau, Jack Lemmon, Ann-Margret]",[Howard Deutch],[Mark Steven Johnson]
3,31357,"[Whitney Houston, Angela Bassett, Loretta Devine]",[Forest Whitaker],"[Ronald Bass, Terry McMillan]"
4,11862,"[Steve Martin, Diane Keaton, Martin Short]",[Charles Shyer],"[Albert Hackett, Nancy Meyers]"


In [22]:
max_directors_length = df_credits_new['directors'].apply(lambda x: len(x) if isinstance(x, list) else 0)
print("Maximum length of directors lists:")
display(max_directors_length.value_counts())

max_writers_length = df_credits_new['writers'].apply(lambda x: len(x) if isinstance(x, list) else 0)
print("\nMaximum length of writers lists:")
display(max_writers_length.value_counts())

Maximum length of directors lists:


directors
1    15417
2     1168
0      203
3      186
Name: count, dtype: int64


Maximum length of writers lists:


writers
1    7400
2    4637
0    2694
3    1482
4     489
5     180
6      59
7      33
Name: count, dtype: int64

3. Remove Duplicates in ID

In [23]:
# Duplicate ID Analysis
duplicate_ids_count = df_credits_new['id'].duplicated(keep=False).sum()
print(f"Number of duplicate ids in df_credits_new: {duplicate_ids_count}")
duplicate_rows = df_credits_new[df_credits_new['id'].duplicated(keep=False)]

# Group by 'id' and compare the lists for cast, directors, and writers
for duplicate_id, group in duplicate_rows.groupby('id'):
    # Check if all rows for this id have the same values in 'cast', 'directors', and 'writers'
    differing_columns = []
    
    for column in ['cast', 'directors', 'writers']:
        # Convert lists to strings for comparison
        if group[column].apply(lambda x: str(x)).nunique() > 1:
            differing_columns.append(column)

    if not differing_columns:
        print(f"All rows for id {duplicate_id} have the same values in 'cast', 'directors', and 'writers'.")
    else:
        print(f"Rows for id {duplicate_id} differ in the following columns: {', '.join(differing_columns)}.")

Number of duplicate ids in df_credits_new: 28
All rows for id 11115 have the same values in 'cast', 'directors', and 'writers'.
All rows for id 116723 have the same values in 'cast', 'directors', and 'writers'.
All rows for id 123634 have the same values in 'cast', 'directors', and 'writers'.
All rows for id 14788 have the same values in 'cast', 'directors', and 'writers'.
All rows for id 15028 have the same values in 'cast', 'directors', and 'writers'.
All rows for id 152795 have the same values in 'cast', 'directors', and 'writers'.
All rows for id 18440 have the same values in 'cast', 'directors', and 'writers'.
All rows for id 187156 have the same values in 'cast', 'directors', and 'writers'.
All rows for id 199591 have the same values in 'cast', 'directors', and 'writers'.
All rows for id 24026 have the same values in 'cast', 'directors', and 'writers'.
All rows for id 69234 have the same values in 'cast', 'directors', and 'writers'.
All rows for id 77221 have the same values in '

In [24]:
print("-- Original Shape of New Credits: ", df_credits_new.shape)
df_credits_new = df_credits_new.drop_duplicates(subset=['id'], keep='first')
print("-- Shape of New Credits after removing duplicates: ", df_credits_new.shape)

-- Original Shape of New Credits:  (16974, 4)
-- Shape of New Credits after removing duplicates:  (16960, 4)


### **Feature Engg with Keywords Dataset**

1. Merge with Common IDs

In [25]:
df_keywords_new = df_keywords.copy()
df_keywords_new['id'] = df_keywords_new['id'].astype(str)
df_keywords_new['keywords'] = df_keywords_new['keywords'].fillna('[]').apply(literal_eval)

print("-- Original Shape of Keywords:", df_keywords_new.shape)
df_keywords_new = pd.merge(df_all_id_filtered[['id']].drop_duplicates(keep='first'), df_keywords_new, left_on='id', right_on='id', how='inner')
print("-- Shape of Merged Keywords and All Common IDs (New Keywords):", df_keywords_new.shape)

-- Original Shape of Keywords: (46419, 2)
-- Shape of Merged Keywords and All Common IDs (New Keywords): (17216, 2)


2. Processing Keywords Column

In [26]:
df_keywords_new['keywords'] = df_keywords_new['keywords'].apply(
    lambda x: [i['name'] for i in x] if isinstance(x, list) else []
    )
df_keywords_new.head()

,id,keywords
0,862,"[jealousy, toy, boy, friendship, friends, rivalry, boy next door, new toy, toy comes to life]"
1,8844,"[board game, disappearance, based on children's book, new home, recluse, giant insect]"
2,15602,"[fishing, best friend, duringcreditsstinger, old men]"
3,31357,"[based on novel, interracial relationship, single mother, divorce, chick flick]"
4,11862,"[baby, midlife crisis, confidence, aging, daughter, mother daughter relationship, pregnancy, contraception, gynecologist]"


3. Removing Duplicates in ID

In [27]:
# Duplicate ID Analysis
duplicate_ids_count = df_keywords_new['id'].duplicated(keep=False).sum()
print(f"Number of duplicate ids in New Keywords: {duplicate_ids_count}")
duplicate_rows = df_keywords_new[df_keywords_new['id'].duplicated(keep=False)]

# Group by 'id' and compare the lists for keywords
for duplicate_id, group in duplicate_rows.groupby('id'):
    # Check if all rows for this id have the same values in 'keywords'
    differing_columns = []
    
    for column in ['keywords']:
        # Convert lists to strings for comparison
        if group[column].apply(lambda x: str(x)).nunique() > 1:
            differing_columns.append(column)

    if not differing_columns:
        pass
        #print(f"All rows for id {duplicate_id} have the same values in 'keywords'.")
    else:
        print(f"Rows for id {duplicate_id} differ in the following columns: {', '.join(differing_columns)}.")

Number of duplicate ids in New Keywords: 512


In [28]:
print("-- Original Shape of Keywords New:", df_keywords_new.shape)
df_keywords_new = df_keywords_new.drop_duplicates(subset=['id'], keep='first')
print(f"-- Shape of Keywords New after removing duplicates:", df_keywords_new.shape)

-- Original Shape of Keywords New: (17216, 2)
-- Shape of Keywords New after removing duplicates: (16960, 2)


In [29]:
# Calculate the length of the keywords column
df_keywords_new['keywords_len'] = df_keywords_new['keywords'].apply(lambda x: len(x) if isinstance(x, list) else 0)
length_frequency = df_keywords_new['keywords_len'].value_counts().sort_index()
df_keywords_new = df_keywords_new.drop(columns=['keywords_len'])
print("Frequency of rows by length of keywords:")
print(length_frequency)

Frequency of rows by length of keywords:
keywords_len
0     4061
1     2246
2     1468
3     1497
4     1406
5     1312
6      978
7      815
8      550
9      460
10     366
11     317
12     260
13     199
14     174
15     157
16     125
17     113
18      80
19      67
20      54
21      53
22      42
23      38
24      24
25      15
26      19
27      15
28       7
29       7
30       4
31       7
32       1
33       2
34       1
36       1
37       4
38       2
40       1
41       3
43       1
44       1
46       1
48       1
50       1
60       2
65       1
97       1
Name: count, dtype: int64


### **Feature Engg with Movies Dataset**

1. Keep only Specific Columns

In [30]:
df_movies_new = df_movies_filtered.copy()
df_movies_new.columns

Index(['id', 'popularity', 'vote_count', 'adult', 'belongs_to_collection',
       'budget', 'genres', 'homepage', 'imdb_id', 'original_language',
       'original_title', 'overview', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average'],
      dtype='object')

In [31]:
# Keep only the specified columns
columns_to_keep = ['id', 'release_date', 'title', 'overview', 'genres', 'popularity', 'vote_count', 'vote_average', 'poster_path']
df_movies_new = df_movies_new[columns_to_keep]

print("Shape of New Movies after keeping specified columns:", df_movies_new.shape)
df_movies_new.info()

Shape of New Movies after keeping specified columns: (16960, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16960 entries, 0 to 16959
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            16960 non-null  object        
 1   release_date  16960 non-null  datetime64[ns]
 2   title         16960 non-null  object        
 3   overview      16960 non-null  object        
 4   genres        16960 non-null  object        
 5   popularity    16960 non-null  float64       
 6   vote_count    16960 non-null  int32         
 7   vote_average  16960 non-null  float64       
 8   poster_path   16891 non-null  object        
dtypes: datetime64[ns](1), float64(2), int32(1), object(5)
memory usage: 1.1+ MB


2. Processing Genres Column

In [32]:
df_movies_new['genres'] = df_movies_new['genres'].fillna('[]').apply(literal_eval)
df_movies_new['genres'] = df_movies_new['genres'].apply(
    lambda x: [i['name'] for i in x] if isinstance(x, list) else []
    )
display(df_movies_new[['id', 'genres']].head())

,id,genres
0,100,"[Comedy, Crime]"
1,100024,"[Horror, Thriller]"
2,10003,"[Thriller, Action, Romance, Science Fiction, Adventure]"
3,100032,"[Drama, Action]"
4,10004,"[Drama, Fantasy, Horror, Thriller, Mystery]"


In [33]:
from collections import Counter

empty_genres_count = df_movies_new['genres'].apply(lambda x: len(x) == 0).sum()
print("Number of rows with empty list in genres:", empty_genres_count)

# Flatten the list of genres
all_genres = [genre for genres_list in df_movies_new['genres'] for genre in genres_list]
genre_frequency = Counter(all_genres)
genre_frequency_df = pd.DataFrame(genre_frequency.items(), columns=['Genre', 'Frequency']).sort_values(by='Frequency', ascending=False)
print("\nGenre frequency:")
display(genre_frequency_df)

Number of rows with empty list in genres: 302

Genre frequency:


,Genre,Frequency
8,Drama,7027
0,Comedy,5217
3,Thriller,3927
4,Action,2753
5,Romance,2442
2,Horror,2195
15,Documentary,2144
1,Crime,1634
12,Family,1507
7,Adventure,1488


In [34]:
zero_popularity_count = (df_movies_new['popularity'] == 0).sum()
zero_vote_count = (df_movies_new['vote_count'] == 0).sum()
zero_vote_average = (df_movies_new['vote_average'] == 0).sum()

print(f"Number of rows with 0 popularity: {zero_popularity_count}")
print(f"Number of rows with 0 vote_count: {zero_vote_count}")
print(f"Number of rows with 0 vote_average: {zero_vote_average}")

Number of rows with 0 popularity: 2
Number of rows with 0 vote_count: 510
Number of rows with 0 vote_average: 533


3. Processing Overviews Column

HTML Tags in Overviews

In [35]:
# Find rows where 'overview' contains HTML tags
overview_rows = df_movies_new[df_movies_new['overview'].str.contains(r'https?://\S+|www\.\S+', na=False)]
overview_rows

,id,release_date,title,overview,genres,popularity,vote_count,vote_average,poster_path
1837,13016,2007-06-01,Zeitgeist,"Zeitgeist: the Movie is a 2007 documentary film by Peter Joseph examining possible historical and modern conspiracies surrounding Christianity, the September 11, 2001 terrorist attacks and the Federal Reserve bank. It was officially released online on June 18, 2007 on www.ZeitgeistMovie.com","[Documentary, History]",4.777597,173,7.0,/wYMpPrFXLDjxn48mXm3OiaC9Drq.jpg
14230,60965,2008-12-09,The Legend of Bloody Mary,"Ryan has been plagued with nightmares since the night his sister Amy went missing 8 years earlier after playing the game ""Bloody Mary."" Amy had stumbled onto a website on the Internet (www.marked4mary.com) about a witch called Bloody Mary and a game to summon her evil spirit. Now a senior in college, Ryan is reaching a mental breaking point from the years of stress and guilt from his sisters disappearance. His girlfriend Rachel frustrated herself at Ryan's emotional distance and self pity, calls for help to a former professor of Ryan's, Father O'Neal. Father O'Neal is both a priest and a archaeologist who decides to help Ryan end his tormenting grief by using his detective skills and wit to figure out what exactly happened to Ryan's sister, and uncover the truth to the Legend of Bloody Mary","[Horror, Mystery, Thriller]",1.076888,5,3.8,/x1kIVaqFOZkTV9zlHIUYsVnqzn8.jpg
16018,87835,2011-10-28,Inkubus,"Inkubus tells the story of a skeleton crew working the final shift at a soon to be demolished police station. The night takes a gruesome turn when the demon, Inkubus, calmly walks into the station holding the severed head of a murdered girl. Inkubus toys with the crew, allowing himself to be restrained, and begins to proudly confess to his litany of crimes, some dating back to the Middle Ages. Why? Inkubus has a score to settle with the one detective that almost put him away some thirteen years ago. To their dismay, the cops quickly become pawns in Inkubus' brutal crowning achievement of murder, gore, and mayhem. They ultimately realize it's his world, they only die in it. Written by Anonymous (www.imdb.com)",[Horror],0.460958,11,4.9,/h3jQubPFth8U7GovTQfZZlTUGgs.jpg


In [36]:
# Define the parts to remove for each id
removals = {
    "292947": "It's available from the www.anybodylisten.com site. ",
    "13016": "It was officially released online on June 18, 2007 on www.ZeitgeistMovie.com",
    "78263": "(http://www.merchantivory.com/custody.html)",
    "60965": "(www.marked4mary.com) ",
    "87835": "Written by Anonymous (www.imdb.com)"
}

# Iterate through the removals dictionary and update the 'overview' column
for movie_id, text_to_remove in removals.items():
    df_movies_new.loc[df_movies_new['id'] == movie_id, 'overview'] = df_movies_new.loc[df_movies_new['id'] == movie_id, 'overview'].str.replace(text_to_remove, '', regex=False)

# Find rows where 'overview' contains HTML tags
overview_rows = df_movies_new[df_movies_new['overview'].str.contains(r'https?://\S+|www\.\S+', na=False)]
overview_rows


,id,release_date,title,overview,genres,popularity,vote_count,vote_average,poster_path


@ Symbol in Overview

In [37]:
# Find rows where 'overview' contains the "@" symbol
at_symbol_rows = df_movies_new[df_movies_new['overview'].str.contains(r'@', na=False)]
at_symbol_rows

,id,release_date,title,overview,genres,popularity,vote_count,vote_average,poster_path
11560,394063,2016-04-30,Chris Hardwick: Funcomfortable,"The “@midnight” host makes things very funcomfortable for the packed house at The Palace of Fine Arts in San Francisco as he explores awkward and sometimes super creepy memories from both childhood and today. With “the energy of SpongeBob dipped in cocaine water,” Hardwick delves into dealing with anxieties, finds the humor in joining the “Dead Dad Club,” and shares deeply personal anecdotes that most people would be too embarrassed to say out loud.",[Comedy],0.440991,3,7.0,/ioEvDcyimIvKBo8TIIjSCOnC9Em.jpg
13312,50043,1998-06-12,Cousin Bette,"Cousin Bette is a poor and lonely seamstress, who, after the death of her prominent and wealthy sister, tries to ingratiate herself into lives of her brother-in-law, Baron Hulot, and her niece, Hortense Hulot. Failing to do so, she instead finds solace and company in a handsome young sculptor she saves from starvation. But the aspiring artist soon finds love in the arms of another woman, Hortense, leaving Bette a bitter spinster. Bette plots to take revenge on the family who turned her away and stole her only love. With the help of famed courtesan Jenny Cadine she slowly destroys the lives of those who have scorned her. Written by CherylC894@aol.com","[Comedy, Drama, Romance]",1.304145,10,5.9,/rkuPUrB4PM6Ob9IvniEpNDO5hvJ.jpg


In [38]:
# Define the parts to remove for each id
removals = {
    "50043": " Written by CherylC894@aol.com",
    "170838": " Written by gavin@sunny_deol2009@yahoo.com",
    "219808": " Written by Philip Cook philcook@eaglefilms.com"
}

# Iterate through the removals dictionary and update the 'overview' column
for movie_id, text_to_remove in removals.items():
    df_movies_new.loc[df_movies_new['id'] == movie_id, 'overview'] = df_movies_new.loc[df_movies_new['id'] == movie_id, 'overview'].str.replace(text_to_remove, '', regex=False)

df_movies_new = df_movies_new[df_movies_new['id'] != "348858"]

# Find rows where 'overview' contains the "@" symbol
at_symbol_rows = df_movies_new[df_movies_new['overview'].str.contains(r'@', na=False)]
at_symbol_rows

,id,release_date,title,overview,genres,popularity,vote_count,vote_average,poster_path
11560,394063,2016-04-30,Chris Hardwick: Funcomfortable,"The “@midnight” host makes things very funcomfortable for the packed house at The Palace of Fine Arts in San Francisco as he explores awkward and sometimes super creepy memories from both childhood and today. With “the energy of SpongeBob dipped in cocaine water,” Hardwick delves into dealing with anxieties, finds the humor in joining the “Dead Dad Club,” and shares deeply personal anecdotes that most people would be too embarrassed to say out loud.",[Comedy],0.440991,3,7.0,/ioEvDcyimIvKBo8TIIjSCOnC9Em.jpg


Newlines in Overview

In [39]:
# Find rows where 'overview' contains HTML tags
overview_rows = df_movies_new[df_movies_new['overview'].str.contains(r'\s{2,}', na=False)]
overview_rows.head()

,id,release_date,title,overview,genres,popularity,vote_count,vote_average,poster_path
27,100272,2011-05-06,Harold's Going Stiff,"Lonely pensioner, Harold Gimble, has become the first man to suffer from a new neurological disease that is slowly turning him into a zombie-like state. Harold’s hermit-like existence is shaken up when a vivacious nurse, Penny Rudge, is sent along to alleviate his stiffness. Her ‘special’ massage techniques work a treat on Harold and they become close friends. Harold agrees to trial a possible cure for the disease at a private institute, and the initial results are excellent. But the following day he’s in an even worse state than before, and after Penny accidentally overhears the doctor’s dire prognosis for Harold she decides to sneak him away the next morning. Word soon gets out, and before long a small group of bloodthirsty thugs are pursuing Harold and Penny across dramatic moorlands in the hope of a kill.","[Comedy, Drama, Horror]",0.913500,8,4.3,/oBeRQjnngmSqITiBhZ9dWmSAqLJ.jpg
106,10139,2008-11-26,Milk,"The story of California's first openly gay elected official, Harvey Milk, who became an outspoken agent for change, seeking equal rights and opportunities for all. His great love for the city and its people brought him backing from young and old, straight and gay, alike – at a time when prejudice and violence against gays was openly accepted as the norm.","[History, Drama]",10.680121,628,7.1,/sFYmJJSVXqfqPI79tx4UG4IR5Jb.jpg
109,101449,2012-04-11,Jim Gaffigan: Mr. Universe,"Recorded at the Warner Theater in Washington, DC on February 25th, 2012, Mr Universe follows in the model of Louis CK's Live At The Beacon Theater and has been made exclusively available for online purchase only, directly from Gaffigan's website. This 75-minute special contains more of Gaffigan's trademark, surreal humor on food, life, his nit-picky fans, and whales in denial. Also, a part of the proceeds from sales of Mr. Universe will go to the Bob Woodruff Foundation, a organization which assists veterans and their families as servicemen return to civilian life.","[Comedy, TV Movie]",1.047845,20,7.2,/hEwxqaWY7fptXNOeXi8U6LRwA2k.jpg
112,10149,1995-06-09,Smoke,"Writer Paul Benjamin is nearly hit by a bus when he leaves Auggie Wren's smoke shop. Stranger Rashid Cole saves his life, and soon middle-aged Paul tells homeless Rashid that he wouldn't mind a short-term housemate. Still grieving over his wife's murder, Paul is moved by both Rashid's quest to reconnect with his father and Auggie's discovery that a woman who might be his daughter is about to give birth.","[Comedy, Drama]",5.552946,115,7.2,/dku3aLprH3j1ilSsubW9Pbgfe0y.jpg
122,101609,2009-11-12,Pax Americana and the Weaponization of Space,"Star Wars is no longer science fiction The prospect of Earth being ruled from space is no longer science-fiction. The dream of the original Dr. Strangelove, Wernher von Braun (from Nazi rocket-scientist to NASA director) has survived every US administration since WW2 and is coming to life. Today the technology exists to weaponize space, a massive American industry thrives, and nations are maneuvering for advantage. PAX AMERICANA tackles this pivotal moment. Are war machines already orbiting Earth? Can treaties keep space weapons-free? Must the World capitulate to one super-cop on the global beat? With startling archival footage and unprecedented access to US Air Force Space Command, this elegant, forceful documentary reveals the state of play through generals, space-policy analysts, politicians, diplomats, peace activists, and hawks.",[Documentary],0.062185,0,0.0,/5UjzOOVVprJUmCWu9DFrIGq96iC.jpg


In [40]:
import re

# Function to clean whitespace
def clean_whitespace(text):
    if not isinstance(text, str):
        return ""
    return re.sub(r'\s+', ' ', text).strip()
df_movies_new['overview'] = df_movies_new['overview'].apply(clean_whitespace)

# Find rows where 'overview' contains HTML tags
overview_rows = df_movies_new[df_movies_new['overview'].str.contains(r'\s{2,}', na=False)]
overview_rows

,id,release_date,title,overview,genres,popularity,vote_count,vote_average,poster_path


HTML

In [41]:
# Find rows where Overview contains HTML tags
overview_rows = df_movies_new[df_movies_new['overview'].str.contains(r'<.*?>', na=False)]
overview_rows

,id,release_date,title,overview,genres,popularity,vote_count,vote_average,poster_path


### **Feature Engg with Ratings Dataset**

1. Merging With Common IDs and Remove movieId

In [42]:
df_ratings_new = df_ratings_filtered.copy()
df_ratings_new['movieId'] = df_ratings_new['movieId'].astype(str)
df_ratings_new['userId'] = df_ratings_new['userId'].astype(str)
print("-- Original Shape of Filtered Ratings:", df_ratings_new.shape)

df_ratings_new = pd.merge(df_all_id_filtered, df_ratings_new, left_on='movieId', right_on='movieId', how='inner')
df_ratings_new['id'] = df_ratings_new['id'].astype(str)
df_ratings_new = df_ratings_new.drop(columns=['movieId'])
print("-- Shape of Merged Filtered Ratings and All Common IDs (New Ratings):", df_ratings_new.shape)

df_ratings_new.info()

-- Original Shape of Filtered Ratings: (26024289, 4)
-- Shape of Merged Filtered Ratings and All Common IDs (New Ratings): (18062187, 4)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18062187 entries, 0 to 18062186
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   id         object 
 1   userId     object 
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 551.2+ MB


2. Drop Duplicates

In [43]:
print("-- Original Shape of New Ratings:", df_ratings_new.shape)
df_ratings_new = df_ratings_new.drop_duplicates()
print("-- Shape of New Ratings after removing fully duplicate rows:", df_ratings_new.shape)

-- Original Shape of New Ratings: (18062187, 4)
-- Shape of New Ratings after removing fully duplicate rows: (18062186, 4)


3. Process Timestamp to Datetime

In [44]:
# Convert the entire 'timestamp' column to datetime objects
df_ratings_new['date'] = pd.to_datetime(df_ratings_new['timestamp'], unit='s')
df_ratings_new['date'] = df_ratings_new['date'].dt.date
df_ratings_new = df_ratings_new.drop(columns=['timestamp'])

# Find the minimum and maximum dates in the 'date' column of df_ratings
print("Minimum date in 'date' column:", df_ratings_new['date'].min())
print("Maximum date in 'date' column:", df_ratings_new['date'].max())

display(df_ratings_new.head())

Minimum date in 'date' column: 1995-01-09
Maximum date in 'date' column: 2017-08-04


,id,userId,rating,date
0,862,8,4.0,2002-02-11
1,862,9,4.5,2004-01-11
2,862,12,4.0,1999-11-29
3,862,20,4.0,2013-05-12
4,862,24,4.0,2001-01-19


4. Analyze Duplicates in id and userId based on date and rating

In [45]:
# Count the number of duplicate ids in df_ratings_new
duplicate_ids_count = df_ratings_new[['id', 'userId']].duplicated(keep=False).sum()
print("Number of duplicate ids in df_ratings_new:", duplicate_ids_count)

print("-- Original Shape of New Ratings:", df_ratings_new.shape)
df_ratings_new = df_ratings_new.drop_duplicates()

print("-- Shape of New Ratings after removing fully duplicate rows:", df_ratings_new.shape)

Number of duplicate ids in df_ratings_new: 42
-- Original Shape of New Ratings: (18062186, 4)
-- Shape of New Ratings after removing fully duplicate rows: (18062183, 4)


In [46]:
# Get rows with duplicate ids in New Ratings
duplicate_rows = df_ratings_new[df_ratings_new[['id', 'userId']].duplicated(keep=False)]
x=0
r=0
# Group by 'id' and compare the lists for date and rating
for (duplicate_id, user_id), group in duplicate_rows.groupby(['id', 'userId']):
    # Check if all rows for this id have the same values in 'date' and 'rating'
    differing_columns = []

    for column in ['date', 'rating']:
        # Convert lists to strings for comparison
        if group[column].apply(lambda x: str(x)).nunique() > 1:
            differing_columns.append(column)
    
    if not differing_columns:
        #print(f"All rows for id {duplicate_id} have the same values in 'date' and 'rating'.")
        x+=1
        r+=group.shape[0]
    else:
        print(f"id {duplicate_id} + userId {user_id} + rows {group.shape[0]}: {', '.join(differing_columns)}.")
        
print(f"{x} unique (id, userId) pairs with {r} total rows are identical.")

id 14788 + userId 228291 + rows 2: date, rating.
id 14788 + userId 237856 + rows 2: date, rating.
id 14788 + userId 45811 + rows 2: date, rating.
id 15028 + userId 147611 + rows 2: date.
id 15028 + userId 167772 + rows 2: date.
id 15028 + userId 190784 + rows 2: date, rating.
id 15028 + userId 230417 + rows 2: date.
id 15028 + userId 4916 + rows 2: date.
id 15028 + userId 83759 + rows 2: date.
id 15028 + userId 89020 + rows 2: date.
id 15028 + userId 95466 + rows 2: rating.
id 18440 + userId 45811 + rows 2: rating.
id 69234 + userId 103725 + rows 2: rating.
id 69234 + userId 109197 + rows 2: rating.
id 69234 + userId 233503 + rows 2: date, rating.
id 77221 + userId 192319 + rows 2: rating.
id 84198 + userId 228291 + rows 2: date, rating.
id 84198 + userId 45811 + rows 2: date.
0 unique (id, userId) pairs with 0 total rows are identical.


In [47]:
# Get rows with duplicate ids in New Ratings
duplicate_rows = df_ratings_new[df_ratings_new[['id', 'userId']].duplicated(keep=False)]
x=0
r=0
# Group by 'id' and compare the lists for rating only
for (duplicate_id, user_id), group in duplicate_rows.groupby(['id', 'userId']):
    # Check if all rows for this id have the same values in 'rating'
    differing_columns = []
    
    for column in ['rating']:
        # Convert lists to strings for comparison
        if group[column].apply(lambda x: str(x)).nunique() > 1:
            differing_columns.append(column)

    if not differing_columns:
        #print(f"All rows for id {duplicate_id} have the same values in 'rating'.")
        x+=1
        r+=group.shape[0]
    else:
        print(f"id {duplicate_id} + userId {user_id} + rows {group.shape[0]}: {', '.join(differing_columns)}.")

print(f"{x} unique (id, userId) pairs with {r} total rows are identical.")

id 14788 + userId 228291 + rows 2: rating.
id 14788 + userId 237856 + rows 2: rating.
id 14788 + userId 45811 + rows 2: rating.
id 15028 + userId 190784 + rows 2: rating.
id 15028 + userId 95466 + rows 2: rating.
id 18440 + userId 45811 + rows 2: rating.
id 69234 + userId 103725 + rows 2: rating.
id 69234 + userId 109197 + rows 2: rating.
id 69234 + userId 233503 + rows 2: rating.
id 77221 + userId 192319 + rows 2: rating.
id 84198 + userId 228291 + rows 2: rating.
7 unique (id, userId) pairs with 14 total rows are identical.


### **Final Dataset**

In [48]:
print('Credits Shape:', df_credits_new.shape)
print('Keywords Shape:', df_keywords_new.shape)
print('Movies Shape:', df_movies_new.shape)
print('Ratings Shape:', df_ratings_new.shape)

Credits Shape: (16960, 4)
Keywords Shape: (16960, 2)
Movies Shape: (16960, 9)
Ratings Shape: (18062183, 4)


In [49]:
# Merge Credits, Keywords, and Movies DataFrames on 'id'
df_merged = pd.merge(df_credits_new, df_keywords_new, on='id', how='inner')
df_merged = pd.merge(df_merged, df_movies_new, on='id', how='inner')
print('Merged Shape:', df_merged.shape)

# Save the merged DataFrame as a CSV file
output_path = os.path.join('data', 'prepared_movies.csv')
df_merged.to_csv(output_path, index=False)
print(f"Merged DataFrame saved to: {output_path}")

# Save the filtered ratings DataFrame as a CSV file
ratings_output_path = os.path.join('data', 'prepared_ratings.csv')
df_ratings_new.to_csv(ratings_output_path, index=False)
print(f"Filtered Ratings DataFrame saved to: {ratings_output_path}")

Merged Shape: (16960, 13)
Merged DataFrame saved to: data\prepared_movies.csv
Filtered Ratings DataFrame saved to: data\prepared_ratings.csv
